## Tokens Distributions and Predicting the Next Token

In [1]:
!pip install -qU langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 12.2 MB/s eta 0:00:00


In [2]:
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.7 MB/s eta 0:00:00


In [3]:
!pip install -qU langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00


The following code uses LangChain’s `OpenAI` class to load GPT-3’s using `gpt-4o-mini` key to complete the sequence, which results in the answer. Before executing the following code, save your OpenAI key in the “OPENAI_API_KEY” environment variable. Moreover, remember to install the required packages

In [4]:
import os
from google.colab import userdata
# Get the API key from Colab's userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

# Set it as an environment variable
os.environ["OPENAI_API_KEY"] = openai_api_key

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

text = "What would be a good company name for a company that makes colorful socks?"

print(llm(text).content)

Here are some creative name ideas for a company that specializes in colorful socks:

1. **Sock Spectrum**
2. **Vibrant Threads**
3. **ColorPop Socks**
4. **Rainbow Stride**
5. **Sassy Socks Co.**
6. **Chroma Socks**
7. **Happy Feet Fabrics**
8. **Sock It to Me**
9. **Dazzle Socks**
10. **Funky Footwear**

Feel free to mix and match or modify these suggestions to find the perfect fit for your brand!


## Tracking Token Usage

You can use the LangChain library's callback mechanism to track token usage. This is currently implemented only for the OpenAI API:

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain_community.callbacks import get_openai_callback

llm = ChatOpenAI(model_name="gpt-4o-mini", n=2)

with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    print(cb)

<ipython-input-7-dde855ac8e99>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", n=2)


Tokens Used: 48
	Prompt Tokens: 11
		Prompt Tokens Cached: 0
	Completion Tokens: 37
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $2.3849999999999997e-05


## Few-shot learning

**Few-shot learning** is a remarkable ability that allows LLMs to learn and generalize from limited examples. Prompts serve as the input to these models and play a crucial role in achieving this feature. With LangChain, examples can be hard-coded, but dynamically selecting them often proves more powerful, enabling LLMs to adapt and tackle tasks with minimal training data swiftly.

This approach involves using the `FewShotPromptTemplateCopy` class, which takes in a `PromptTemplate` and a list of a few shot examples. The class formats the prompt template with a few shot examples, which helps the language model generate a better response. We can streamline this process by utilizing LangChain's FewShotPromptTemplate to structure the approach:

In [8]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

After creating a template, we pass the example and user query, and we get the results

In [9]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

# load the model
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

chain = few_shot_prompt_template | chat
chain.invoke("What's the meaning of life?")

AIMessage(content="The meaning of life is 42... or maybe it's just finding the perfect slice of pizza!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 98, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-7ef50481-fa22-4cb2-b682-837ac066f11d-0', usage_metadata={'input_tokens': 98, 'output_tokens': 20, 'total_tokens': 118, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Examples with Easy Prompts: Text Summarization, Text Translation, and Question Answering

In the realm of natural language processing, Large Language Models have become a popular tool for tackling various text-based tasks. These models can be promoted in different ways to produce a range of results, depending on the desired outcome.

Setting Up the Environment

To begin, we will need to install the `huggingface_hub` library in addition to previously installed packages and dependencies. Also, keep in mind to create the Huggingface API Key by navigating to Access Tokens page under the account’s Settings. The key must be set as an environment variable with `HUGGINGFACEHUB_API_TOKEN` key.



In [10]:
!pip install -q huggingface_hub

## Creating a Question-Answering Prompt Template

Let's create a simple question-answering prompt template using LangChain

In [11]:
from langchain_core.prompts.prompt import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"

Next, we will use the Hugging Face model `google/flan-t5-large`to answer the question. The `HuggingfaceHub` class will connect to Hugging Face’s inference API and load the specified model.

In [15]:
from langchain import HuggingFaceHub
from langchain.chains import LLMChain

huggingfacehub_api_token = userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Set it as an environment variable
os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingfacehub_api_token

# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0}
)

chain = prompt | hub_llm

# ask the user question about the capital of France
print(chain.invoke(question))

paris


We can also modify the *prompt template* to include multiple questions.

## Asking Multiple Questions

To ask multiple questions, we can either iterate through all questions one at a time or place all questions into a single prompt for more advanced LLMs. Let's start with the first approach:

In [23]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
llm_chain = LLMChain(llm=hub_llm, prompt=prompt)
res = llm_chain.generate(qa)
print( res )

generations=[[Generation(text='paris')], [Generation(text='giraffe')], [Generation(text='nitrogen')], [Generation(text='yellow')]] llm_output=None run=[RunInfo(run_id=UUID('16be5b87-dfba-4db5-96fa-30c0d9b3c477')), RunInfo(run_id=UUID('45847fa4-7be5-4ab8-ad69-05caac097c47')), RunInfo(run_id=UUID('96b9c607-e4aa-43a8-9ae9-b02d27a378c4')), RunInfo(run_id=UUID('00b1a393-5bf6-4844-9a94-98d1769aad72'))] type='LLMResult'


## Text Summarization
Using LangChain, we can create a chain for text summarization. First, we need to set up the necessary imports and an instance of the OpenAI language model:

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)
summarization_chain = summarization_prompt | llm

text = "LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."
summarized_text = summarization_chain.invoke(text)
summarized_text.content

'LangChain offers various modules for building language model applications, allowing for both simple and complex implementations, with the fundamental task being the invocation of a language model on specific input, exemplified by creating a service to generate company names based on their products.'

## Text Translation
It is one of the great attributes of Large Language models that enables them to perform multiple tasks just by changing the prompt. We use the same `llm` variable as defined before. However, pass a different prompt that asks for translating the query from a `source_language` to the `target_language`.

In [47]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=["source_language", "target_language", "text"], template=translation_template)
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)

source_language = "English"
target_language = "French"
text = "How are you"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)
translated_text

'The translation of "How are you" in French is "Comment ça va ?" or simply "Ça va ?" for a more casual approach.'